In [12]:
import pandas as pd
from MKtest import *

In [13]:
d = dict(A=1, B=2, C=3, D=4)

In [14]:
file = "/home/sur/micropopgen/exp/today/mktest/Streptococcus_sp_57750/mk_results.Buccal.mucosa_Tongue.dorsum.txt"
pd.read_csv(file, sep="\t").head()

,gene,contig,start,end,Dn,Ds,Pn,Ps,hg,hg.pval
0,1156433.3.peg.1,CP006776,1134,2262,6,3,29,71,0.238095,0.037896
1,1156433.3.peg.2,CP006776,2271,2427,0,0,2,15,0.187500,0.352381
2,1156433.3.peg.3,CP006776,2531,2897,0,0,3,13,0.285714,0.447368
3,1156433.3.peg.4,CP006776,2935,4101,1,1,3,1,2.000000,1.000000
4,1156433.3.peg.5,CP006776,4162,4562,2,0,13,10,0.424242,0.622079


In [18]:
class ARGS:
    def __init__(self, indir, metadata_file,
                group1, group2, test='hg',
                outfile='mk_results', min_count=1,
                nrows=float('inf'), tables='mk_tables',
                pseudocount=0, permutations=0, seed=5):
        
        self.indir = indir
        self.metadata_file = metadata_file
        self.group1 = group1
        self.group2 = group2
        self.test = test
        self.outfile = outfile
        self.min_count = min_count
        self.nrows = nrows
        self.tables = tables
        self.pseudocount = pseudocount
        self.permutations = permutations
        self.seed = seed


In [19]:
args = ARGS(indir='/home/sur/micropopgen/data/test_data/midas/merged.snps//Veillonella_parvula_57794/',
            metadata_file='/home/sur/micropopgen/data/test_data/midas/map.txt',
            group1='Supragingival plaque',
            group2='Tongue dorsum',
            outfile='mktest/Veillonella_parvula_57794//mk_results.Supragingival.plaque_Tongue.dorsum.txt',
            tables='mktest/Veillonella_parvula_57794//mk_tables.Supragingival.plaque_Tongue.dorsum.txt',
            nrows=2000,
            pseudocount=0,
            permutations=10,
            seed=5)

In [20]:
print("Checking MIDAS files exist")
confirm_midas_merge_files(args)

# Read mapping files
# Create dictionaries that have all the samples per group (Groups),
# and the group to which each sample belongs (Samples)
# Probably should change this to pandas
print("Read metadata")
Samples, Groups = process_metadata_file(args.metadata_file)

print("Calculate MK contingency tables")
MK, Genes = calculate_contingency_tables(Samples, Groups, args)


Checking MIDAS files exist
	All files found
Read metadata
Calculate MK contingency tables
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1883
Number of genes: 3
	Chose sites based on depth in groups to compare
Number of sites: 1830
Number of genes: 3
Sites with counts: 1830
	Read frequencies and calculate
Processing snp_freq.txt
Number of sites: 1830
Number of genes: 3
Sites with counts: 1830
Genes with MK: 3


In [21]:
if args.permutations > 0:
        MK = [MK]
        print("Permuting")
        print("Seed is {}".format(str(args.seed)))
        for i in range(args.permutations):
            Sp, Gp = process_metadata_file(args.metadata_file, permute=True)
            MK.append(calculate_contingency_tables(Sp, Gp, args))

Permuting
Seed is 5
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1883
Number of genes: 3
	Chose sites based on depth in groups to compare
Number of sites: 1883
Number of genes: 3
Sites with counts: 1883
	Read frequencies and calculate
Processing snp_freq.txt
Number of sites: 1883
Number of genes: 3
Sites with counts: 1883
Genes with MK: 3
	Read snps_info.txt
============HEADERs============
>Site id: site_id
>Contig: ref_id
>Position: ref_pos
>Ref allele: ref_allele
>Major allele: major_allele
>Minor allele: minor_allele
>Locus type: locus_type
>Gene id: gene_id
>Aminoacids: amino_acids

Number of sites: 1883
Number of genes: 3
	Chose sites based on depth in groups to compare
Number of sites: 1883
Number of genes: 3
Sites with counts: 1883
	Read frequencies and calc

In [32]:
# Get list of tests to perform
test = args.test
supported_tests = ['NI', 'ratio', 'hg',
                   'G', 'G_Yates', 'G_Williams_p',
                   'alpha']
if test == 'all':
    test = supported_tests
elif test not in supported_tests:
    raise ValueError("Test not supported")
else:
    test = [test]

# Create header
header_base = ['gene', 'contig', 'start', 'end',
               'Dn', 'Ds', 'Pn', 'Ps']
pval_list = [''.join([t, '.pval']) for t in test]
header = header_base + test + pval_list
print(test)
print(pval_list)


['hg']
['hg.pval']


In [29]:
mk

{'686660.3.peg.1': <MKtest.MKtest at 0x7f96002e3c50>,
 '686660.3.peg.2': <MKtest.MKtest at 0x7f96002e3a20>,
 '686660.3.peg.3': <MKtest.MKtest at 0x7f96002e3748>}